Para entrar no modo apresentação, execute a seguinte célula e pressione `-`

In [ ]:
%reload_ext slide

<span class="notebook-slide-start"/>

# API v4

Este notebook apenas apresenta a API v4.

Anteriormente, o minicurso abordou a API v3 do GitHub, que utiliza REST. Agora, o minicurso abordará a API v4, que usa GraphQL (https://developer.github.com/v4/).

Antes de qualquer coisa, vamos iniciar o servidor de proxy, caso ele esteja fechado:
```bash
python proxy.py https://api.github.com/
```


Além do servidor de proxy, precisamos carregar o token e preparar a função de autenticação.  <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
from ipywidgets import FileUpload, interact
@interact(files=FileUpload())
def set_token(files={}):
    global token
    if files:
        for key, values in files.items():
            token = values['content'].decode("utf-8").strip()
            print("Token Loaded!")

In [ ]:
import requests

def token_auth(request):
    request.headers["User-Agent"] = "Minicurso" # Necessário
    request.headers["Authorization"] = "token {}".format(token)
    return request

Agora podemos tentar conectar na API v4 e verificar se a autenticação funcionou. Note que usamos POST e URL original é https://api.github.com/graphql. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
SITE = "http://localhost:5000/" # ou https://api.github.com

query = """
{
  rateLimit {
    limit
    cost
    remaining
    resetAt
  }
}
"""

response = requests.post(SITE + "graphql", json={'query': query}, auth=token_auth)
response.status_code

In [ ]:
response.json()

A consulta com a API v4 é um pouco mais verbosa, porém existe uma única URL de acesso e o resultado vem exatamente o que consultamos. <span class="notebook-slide-scroll" data-count="-1"/>

A seguir temos um exemplo de uma consulta quase completa em relação ao que fizemos na APIv3. <span class="notebook-slide-extra" data-count="1"/>

In [ ]:
query = """
query {
  repository(owner:"gems-uff", name:"sapos") {
      stargazers {
          totalCount
      }
      forks {
          totalCount
      }
      watchers {
          totalCount
      }
      primaryLanguage {
          name
      }
      open_issues: issues(states:OPEN, first:100) {
          totalCount
          edges {
              node {
                  number
                  closedAt
                  createdAt
                  labels(first:100) {
                    edges { 
                      node {
                        name
                      }
                    }
                    pageInfo {
                      startCursor
                      hasNextPage
                      endCursor
                    }
                  }
              }
          }
          pageInfo {
            startCursor
            hasNextPage
            endCursor
          }
      }
      closed_issues: issues(states:CLOSED, first:100) {
          totalCount
          edges {
              node {
                  number
                  closedAt
                  createdAt
                  labels(first:100) {
                    edges { 
                      node {
                        name
                      }
                    }
                    pageInfo {
                      startCursor
                      hasNextPage
                      endCursor
                    }
                  }
              }
          }
          pageInfo {
            startCursor
            hasNextPage
            endCursor
          }
      }
      mentionableUsers(first:100) {
          edges {
              node {
                  login
              }
          }
          pageInfo {
            startCursor
            hasNextPage
            endCursor
          }
      }
  }
  
}
"""

response = requests.post(SITE + "graphql", json={'query': query}, auth=token_auth)
print(response.status_code)
data = response.json()
data

Uma única consulta é capaz de retornar boa parte das informações que precisamos.

Mas ATENÇÃO! Paginação ainda é necessária e é feita com os argumentos `first:100` e `after:{endCursor}`. <span class="notebook-slide-extra" data-count="3"/>

In [ ]:
data["data"]["repository"]["closed_issues"]["pageInfo"]

In [ ]:
query_base = """
query {
  repository(owner:"gems-uff", name:"sapos") {
      closed_issues: issues(states:CLOSED, first:100, after:"%s") {
          totalCount
          edges {
              node {
                  number
                  closedAt
                  createdAt
                  labels(first:100) {
                    edges { 
                      node {
                        name
                      }
                    }
                    pageInfo {
                      startCursor
                      hasNextPage
                      endCursor
                    }
                  }
              }
          }
          pageInfo {
            startCursor
            hasNextPage
            endCursor
          }
      }
  }
}
"""
query = query_base % (data["data"]["repository"]["closed_issues"]["pageInfo"]['endCursor'], )

response = requests.post(SITE + "graphql", json={'query': query}, auth=token_auth)
print(response.status_code)
data2 = response.json()
data2

In [ ]:
data2["data"]["repository"]["closed_issues"]["pageInfo"]

Mais uma página. <span class="notebook-slide-extra" data-count="2"/>

In [ ]:
query = query_base % (data2["data"]["repository"]["closed_issues"]["pageInfo"]['endCursor'], )

response = requests.post(SITE + "graphql", json={'query': query}, auth=token_auth)
print(response.status_code)
data3 = response.json()
data3

In [ ]:
data3["data"]["repository"]["closed_issues"]["pageInfo"]

Foi a última. <span class="notebook-slide-scroll" data-position="-1"/>

### Schema

O schema da API v4 pode ser encontrado na documentação: https://developer.github.com/v4/object/repository/

Além disso, é possível fazer consultas para obter o schema. <span class="notebook-slide-extra" data-count="1"/>


In [ ]:
SITE = "http://localhost:5000/" # ou https://api.github.com

query = """
query {
  __type(name: "Repository") {
    name
    kind
    description
    fields {
      name
      description
    }
  }
}
"""

response = requests.post(SITE + "graphql", json={'query': query}, auth=token_auth)
print(response.status_code)
response.json()

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;


&nbsp;

&nbsp;

&nbsp;

